In [1]:
from IPython.display import display, Markdown, Latex
import os
import os.path
import pandas
import functools

from ee import EeVal
from ee.formatting import e12, e24
#from ee.kicad.bom.io import read_bom
from ee.kicad import read_schematic, read_schematics, to_pandas

%load_ext autoreload
%autoreload 2


In [2]:
bom_file = "alpha.sch"
components = to_pandas(read_schematics(bom_file))

### LED resistors

In [3]:
leds = components[components.role == "LED"]

def resistance(led):
    i_target = EeVal(led.req_I).value
    v_resistor = EeVal(led.req_Vin).value - EeVal(led.req_Vf).value
    r_exact = v_resistor/i_target
    r = e12.closest(r_exact)
    i_exact = EeVal("{} A".format(v_resistor/r))
    return pandas.Series({"r": r, "r_exact": r_exact, "i": i_exact})

resistances = leds.apply(resistance, axis='columns', reduce=False)

for ref, led in leds.join(resistances).iterrows():
    display(Markdown("#### LED {}".format(ref)))
    display(Latex("""
$$
\\begin{{align}}
V_{{forward}} &= {} \\\\
I             &= {} \\\\
R_{{exact}}   &= {} \Omega \\\\
\\\\
R_{{e12}}     &= {} \Omega \\\\
I_{{LED}}     &= {} A
\\end{{align}}
$$
""".format(led.req_Vf, led.req_I, led.r_exact, led.r, led.i)))

#### LED D2

<IPython.core.display.Latex object>

#### LED D4

<IPython.core.display.Latex object>

#### LED D6

...

<IPython.core.display.Latex object>

#### LED D22

<IPython.core.display.Latex object>

#### LED D23

<IPython.core.display.Latex object>

In [4]:
components = components.join(resistances)

In [5]:
components[components.ref_type == "R"][["value"]] \
    .sort_values("value")

,value
ref,
R21,100k
R1,10k
R20,10k
R19,10k
R18,10k
R17,10k
R16,10k
R15,10k
R13,10k
